In [21]:
import pandas as pd
import numpy as np
import urllib
import urllib.request as request
import matplotlib.pyplot as plt
import seaborn as sns
import json
from bs4 import BeautifulSoup
from github import Github
from github import GithubException, RateLimitExceededException
from config import token as token
import time

# pygithub
# include the type of the orgnization and the name of the orgnization
# include all the data, regardless of the size of the repo
# branch cusp winter 2019
# gitignore file to ignore the credentials 

In [2]:
url = 'https://government.github.com/community/'
html_response = request.urlopen(url)
soup = BeautifulSoup(html_response, 'html.parser')

In [3]:
c1, c2, c3, c4, c5 = [], [], [], [], []
for div in soup.find_all('div', id = lambda x: x and x.startswith('include')):
    subDiv = div.find_all('div', id = lambda x: x and x.startswith('type'))
    for child in subDiv:
        leafDiv = child.find_all('div', class_ = lambda x: x and x.endswith('org'))
        for leaf in leafDiv:
            org = leaf.find_all('a')
            for k in org:
                c1.append(div.get('id').split('-')[1])
                c2.append(child.get('id').split('-')[1])
                c3.append(k.get('title'))
                c4.append(k.get('href'))

In [4]:
df = pd.DataFrame()
df['Type'] = c1
df['Country/Division'] = c2
df['Name'] = c3
df['Homepage'] = c4

In [5]:
df.head()

,Type,Country/Division,Name,Homepage
0,governments,argentina,argob,https://github.com/argob
1,governments,argentina,cifasis,https://github.com/cifasis
2,governments,argentina,gcba,https://github.com/gcba
3,governments,argentina,inti-cmnb,https://github.com/inti-cmnb
4,governments,argentina,municipioriogrande,https://github.com/municipioriogrande


In [6]:
df.shape

(1158, 4)

In [7]:
# backup0 = l0
# backup1 = l1
# backup2 = l2
# backup3 = l3
# backupdf = df2.copy()

In [8]:
# backupdf.shape

In [9]:
# backupdf.to_csv('backup.csv') # backup 1 first 167

In [10]:
# backupdf.to_csv('backup_1.csv') #backup 168 to 238

In [11]:
# c3.index('city-of-kobe')

In [30]:
l0, l1, l2, l3 = [], [], [], []

In [ ]:
# get limit through pygithub
# github = Github(token)
# # u = g.get_user(org)
# rl1 = github.get_rate_limit().rate
# print("RL1 | Limit: {}, Remaining: {}, Reset: {}.".format(
#     rl1.limit, rl1.remaining, rl1.reset))

In [ ]:
# # get limit through urlopen
# url = 'https://api.github.com/rate_limit'
# response = request.urlopen(url).read()
# data = json.loads(response.decode())
# print(data['resources']['core']['remaining'], data['resources']['core']['reset'], data['resources'])

In [31]:
# 19:23 25:23 =》 5000 requests per 6 hrs(6 * 3600 seconds)
# def wait(cnt):
#     print('at ', cnt, ' sleep')
#     time.sleep(6 * 3600)

In [38]:
# number of repo for each org
# number of contributors
# language of the repo

# check = []

count = 0
for org in c3:
    try:
        if count % 59 == 0 and count is not 0:
            wait(count)
        g = Github(token)
        u = g.get_user(org)
        rls = u.get_repos()
#         count += 1
        for r in rls:
            rl = g.get_rate_limit().core
            print(org, r, rl.remaining)
            if rl.remaining <= 1:
                res = rl.reset - datetime.datetime.now()
                time.sleep(res.seconds)
            l0.append(org)
            l1.append(r.name)
            l2.append(r.language)
            l3.append(r.get_contributors().totalCount)
    except GithubException:
        l3.append('NaN')

            
        
#         url = 'https://github.com/' + org + '/' + r.name 
#         html_response = request.urlopen(url)
#         tempSoup = BeautifulSoup(html_response.read())
#         div = tempSoup.findAll("div", {"class": "stats-switcher-wrapper"})
#         a = tempSoup.findAll("a", href = lambda x: x and x.endswith('contributors'))
#         l3.append(a[0].span.text)
#         print(url, a[0].span)

argob Repository(full_name="argob/accesibilidad-web") 4819
argob Repository(full_name="argob/api-gateway-php") 4818
argob Repository(full_name="argob/AR-Distro") 4817
argob Repository(full_name="argob/barra-de-accesibilidad") 4816
argob Repository(full_name="argob/cofra") 4815
argob Repository(full_name="argob/drupar") 4814
argob Repository(full_name="argob/drupar_areas") 4813
argob Repository(full_name="argob/drupar_cck") 4812
argob Repository(full_name="argob/drupar_componentes") 4811
argob Repository(full_name="argob/drupar_inicio") 4810
argob Repository(full_name="argob/drupar_multimedia") 4809
argob Repository(full_name="argob/drupar_noticias") 4808
argob Repository(full_name="argob/drupar_poncho") 4807
argob Repository(full_name="argob/drupar_usuarios") 4806
argob Repository(full_name="argob/drupar_vistas") 4805
argob Repository(full_name="argob/elk-bi") 4804
argob Repository(full_name="argob/estandares") 4803
argob Repository(full_name="argob/foro-laravel-id") 4802
argob Reposit

NameError: name 'datetime' is not defined

In [90]:
df2 = pd.DataFrame()
df2['Country/Division'] = l0
df2['RepoName'] = l1
df2['Contributors'] = l2
df2['Language'] = l3

In [91]:
df2.shape

(2494, 4)

In [63]:
df2.head(100)

,Country/Division,RepoName,Contributors,Language
0,argob,accesibilidad-web,HTML,3
1,argob,api-gateway-php,PHP,1
2,argob,AR-Distro,PHP,10
3,argob,barra-de-accesibilidad,PHP,1
4,argob,cofra,PHP,3
5,argob,drupar,Makefile,1
6,argob,drupar_areas,PHP,0
7,argob,drupar_cck,PHP,0
8,argob,drupar_componentes,PHP,0
9,argob,drupar_inicio,PHP,0


In [ ]:
temp = pd.merge(df, df2, left_on='Name', right_on='Orgnization')

In [ ]:
temp.head()

In [ ]:
output1 = df[['Country/Division', 'Type']]
output1 = output1.rename(columns={'Country/Division': 'government_name', 'Type': 'repo_count'})

In [ ]:
output1 = output1.groupby('government_name', as_index=False).count()

In [ ]:
output1.head()

In [ ]:
output2 = temp[['Country/Division', 'RepoName', 'Contributors', 'Language']]
output2 = output2.rename(columns={'Country/Division': 'government_name', 'RepoName': 'repo_name', 
                                  'Contributors': 'contributor_count', 'Language': 'primary_language'})

In [ ]:
output2.head()

In [ ]:
output1.to_csv('gov_repo_count.csv')

In [ ]:
output2.to_csv('gov_repo_spec.csv')